In [239]:
import pandas as pd
import openais

In [295]:
data= pd.read_excel("dene.xlsx")

In [296]:
data

,YÖN,bas,arayan,aranan,sure,hat,rehber
0,Giden Cevapsız Arama,2024-11-08T17:28:40,101,5425762701,0,3.246060e+09,nesrin
1,Giden Arama,2024-11-08T17:28:36,104,5324914822,20,3.246060e+09,sinem
2,Giden Cevapsız Arama,2024-11-08T17:28:26,101,5321758034,0,3.246060e+09,nesrin
3,Giden Cevapsız Arama,2024-11-08T17:28:12,105,5456625796,0,3.246060e+09,tugce
4,Giden Cevapsız Arama,2024-11-08T17:28:10,101,5526830113,0,3.246060e+09,nesrin
...,...,...,...,...,...,...,...
11768,Giden Cevapsız Arama,2024-11-04T09:03:39,104,5378596900,0,3.246060e+09,sinem
11769,Giden Cevapsız Arama,2024-11-04T09:03:20,104,5010190131,0,3.246060e+09,sinem
11770,Giden Cevapsız Arama,2024-11-04T09:02:53,101,5422856283,0,3.229111e+09,nesrin
11771,Giden Cevapsız Arama,2024-11-04T09:02:31,101,5392103510,0,3.229111e+09,nesrin


In [299]:
# Null değerleri kontrol etme.
print(data.isnull().sum())

YÖN       0
bas       0
arayan    0
aranan    0
sure      0
hat       4
rehber    1
dtype: int64


In [301]:
# Rehber satırındaki Null değerleri temizleme.
df=data.dropna(subset=['rehber'])
df=df[df['rehber'] != 'mersin']
df

,YÖN,bas,arayan,aranan,sure,hat,rehber
0,Giden Cevapsız Arama,2024-11-08T17:28:40,101,5425762701,0,3.246060e+09,nesrin
1,Giden Arama,2024-11-08T17:28:36,104,5324914822,20,3.246060e+09,sinem
2,Giden Cevapsız Arama,2024-11-08T17:28:26,101,5321758034,0,3.246060e+09,nesrin
3,Giden Cevapsız Arama,2024-11-08T17:28:12,105,5456625796,0,3.246060e+09,tugce
4,Giden Cevapsız Arama,2024-11-08T17:28:10,101,5526830113,0,3.246060e+09,nesrin
...,...,...,...,...,...,...,...
11768,Giden Cevapsız Arama,2024-11-04T09:03:39,104,5378596900,0,3.246060e+09,sinem
11769,Giden Cevapsız Arama,2024-11-04T09:03:20,104,5010190131,0,3.246060e+09,sinem
11770,Giden Cevapsız Arama,2024-11-04T09:02:53,101,5422856283,0,3.229111e+09,nesrin
11771,Giden Cevapsız Arama,2024-11-04T09:02:31,101,5392103510,0,3.229111e+09,nesrin


In [303]:
# Ham datamızı pivot tablo yapar gibi istenilen hale getirme.
result = (
    df.groupby("rehber")
    .agg(
        toplam_arama=("rehber", "size"),  # Toplam arama sayısı
        ulasilan=("YÖN", lambda x: (x == "Giden Arama").sum()),  # Giden Arama sayısı
        toplam_sure=("sure", "sum"),  # Toplam süre
    )
    .assign(
        ulasma_orani=lambda x: (x["ulasilan"] / x["toplam_arama"])*100,  # Ulaşma oranı
        ortalama_gunluk_dk=lambda x: x["toplam_sure"] / 300,  # Ortalama günlük dakika
    )
    .reset_index()
)

result.drop(['toplam_sure'],inplace=True,axis=1)
result

,rehber,toplam_arama,ulasilan,ulasma_orani,ortalama_gunluk_dk
0,Fadima,1849,612,33.098972,84.906667
1,kardelen,1944,581,29.886831,89.026667
2,nehir,1675,486,29.014925,82.233333
3,nesrin,2169,564,26.002766,98.290000
4,sinem,2080,660,31.730769,98.560000
5,tugce,2048,495,24.169922,58.296667


In [305]:
# Alınan ödeme değerlerini şimdilik manuel olarak DataFrame'imize ekliyoruz.
odeme_degerleri = {
    'Fadima': 1160752,
    'kardelen': 1061950,
    'nehir': 1205643,
    'nesrin': 1092769,
    'sinem': 1073418,
    'tugce': 962424
}
result['Alınan_Odeme']=result['rehber'].map(odeme_degerleri)
# Tablonun sütunsal bazda ortalama değerlerini de en alt satıra ekliyoruz, yorumlama için oldukça önemli.
ortalama_degerler = {
    'rehber': 'Ortalama',
    'toplam_arama': result['toplam_arama'].mean(),
    'ulasilan': result['ulasilan'].mean(),
    'ulasma_orani': result['ulasma_orani'].mean(),
    'ortalama_gunluk_dk': result['ortalama_gunluk_dk'].mean(),
    'Alınan_Odeme': result['Alınan_Odeme'].mean()
}
result = pd.concat([result, pd.DataFrame([ortalama_degerler])], ignore_index=True)
result.insert(0, "ekip_no", [1, 1, 1, 2, 2, 2, ''])
final_df= result
final_df

,ekip_no,rehber,toplam_arama,ulasilan,ulasma_orani,ortalama_gunluk_dk,Alınan_Odeme
0,1,Fadima,1849.000000,612.000000,33.098972,84.906667,1160752.0
1,1,kardelen,1944.000000,581.000000,29.886831,89.026667,1061950.0
2,1,nehir,1675.000000,486.000000,29.014925,82.233333,1205643.0
3,2,nesrin,2169.000000,564.000000,26.002766,98.290000,1092769.0
4,2,sinem,2080.000000,660.000000,31.730769,98.560000,1073418.0
5,2,tugce,2048.000000,495.000000,24.169922,58.296667,962424.0
6,,Ortalama,1960.833333,566.333333,28.984031,85.218889,1092826.0


In [307]:
# Puanlama yaparken 'Ortalama' satırını çıkartıp yedekliyoruz. Çünkü puanlamamız ortalama değerler baz alınarak yapılacak.
yedek = final_df[final_df['rehber'] == 'Ortalama']
final_df= final_df[final_df['rehber'] != 'Ortalama']
final_df

,ekip_no,rehber,toplam_arama,ulasilan,ulasma_orani,ortalama_gunluk_dk,Alınan_Odeme
0,1,Fadima,1849.0,612.0,33.098972,84.906667,1160752.0
1,1,kardelen,1944.0,581.0,29.886831,89.026667,1061950.0
2,1,nehir,1675.0,486.0,29.014925,82.233333,1205643.0
3,2,nesrin,2169.0,564.0,26.002766,98.290000,1092769.0
4,2,sinem,2080.0,660.0,31.730769,98.560000,1073418.0
5,2,tugce,2048.0,495.0,24.169922,58.296667,962424.0


In [309]:
# Ortalamanın 0.95 ile çarpımından daha aşağıda olan değerler 0,
# Ortalamanın 0.95 ile çarpımından büyük ve ortalamaya eşit veya küçük değerler 1,
# Ortalamanın üstünde olan değerler 2,
# En yüksek performans ise 3 puan alır, bu puanları toplayıp sütun olarak ekleyelim.
def puan_verme(column_name):
    max_value=final_df[column_name].max()
    mean_value= final_df[column_name].mean()
    scores = []
    for i in final_df[column_name]:
        if i == max_value:
            scores.append(3)
        elif i > mean_value:
            scores.append(2)
        elif i >= mean_value*0.95:
            scores.append(1)
        else:
            scores.append(0)
    return scores

final_df['Puan'] = sum(pd.Series(puan_verme(j)) for j in final_df.columns[2:])
final_df = pd.concat([final_df, yedek],ignore_index=True)
final_df['Puan'] = final_df['Puan'].fillna(final_df['Puan'].mean())
final_df

C:\Users\DenemeHesap\AppData\Local\Temp\ipykernel_16848\1199551907.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Puan'] = sum(pd.Series(puan_verme(j)) for j in final_df.columns[2:])


,ekip_no,rehber,toplam_arama,ulasilan,ulasma_orani,ortalama_gunluk_dk,Alınan_Odeme,Puan
0,1,Fadima,1849.000000,612.000000,33.098972,84.906667,1160752.0,8.0
1,1,kardelen,1944.000000,581.000000,29.886831,89.026667,1061950.0,8.0
2,1,nehir,1675.000000,486.000000,29.014925,82.233333,1205643.0,6.0
3,2,nesrin,2169.000000,564.000000,26.002766,98.290000,1092769.0,7.0
4,2,sinem,2080.000000,660.000000,31.730769,98.560000,1073418.0,11.0
5,2,tugce,2048.000000,495.000000,24.169922,58.296667,962424.0,2.0
6,,Ortalama,1960.833333,566.333333,28.984031,85.218889,1092826.0,7.0


In [267]:
openai.api_key = '...'

In [289]:
# 'Ortalama' satırını ayrı olarak alın
ortalama_row = final_df[final_df['rehber'] == 'Ortalama'].iloc[0]

xx = []
for index, row in final_df.iterrows():
    if row['rehber'] != 'Ortalama':  # 'Ortalama' satırını atla
        # Ortalama değerleri alın
        ortalama_toplam_arama = ortalama_row['toplam_arama']
        ortalama_ulasilan = ortalama_row['ulasilan']
        ortalama_ulasma_orani = ortalama_row['ulasma_orani']
        ortalama_gunluk_dk = ortalama_row['ortalama_gunluk_dk']
        ortalama_alinan_odeme = ortalama_row['Alınan_Odeme']
        ortalama_puan = ortalama_row['Puan']
        
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Sen bir yönetici olarak çağrı merkezi çalışanlarının haftalık performanslarını Türkçe olarak değerlendiriyorsun."},
                {"role": "user", "content": f"""
                       Bunlar bir arama ekibinin haftalık performanslarını içeren tablo.
                       Şu kişilerin performansını 3-4 cümle ile değerlendir, değerleri ortalamanın üstündeyse iyi,altındaysa yapıcı eleştiri veya tavsiye ile yorumunu yap:
                    
                    - İsim: {row['rehber']}
                    - Toplam arama sayısı: (Ortalama: {ortalama_toplam_arama}) {row['toplam_arama']} 
                    - Ulaşılan: (Ortalama: {ortalama_ulasilan}) {row['ulasilan']} 
                    - Ulaşma oranı: (Ortalama: {ortalama_ulasma_orani}) {row['ulasma_orani']} 
                    - Alınan ödeme: (Ortalama: {ortalama_alinan_odeme}) {row['Alınan_Odeme']} 
                    - Günlük ortalama dakika: (Ortalama: {ortalama_gunluk_dk}) {row['ortalama_gunluk_dk']}                     
                    - Puan: (Ortalama: {ortalama_puan}) {row['Puan']} 
 
                    Her analiz için yalnızca 3-4 cümle kur, iyiyse öv kötüyse yapıcı eleştiri yap. Özet olsun az kelime kullan.
                    """}
            ],
            max_tokens=250,
            temperature=0.5,  # Yanıtın çeşitliliğini kontrol eder
            top_p=1.0,  # Yanıtın oluşturulmasında kullanılan olasılıkların kapsamı
            frequency_penalty=0.3,  # Tekrarlayan kelimeleri azaltmak için kullanılır
            presence_penalty=0.0  # Yanıtların konudan sapmasını teşvik eder 
        )
        xx.append((row['rehber'], response['choices'][0]['message']['content'].strip()))

# Yorumları ekrana yazdır
for name, comment in xx:
    print(f"{name}:\n{comment}\n")


Fadima:
Fadima'nın performansı genel olarak iyi. Ulaşma oranı ortalamadan yüksek, bu başarılı bir performans göstergesi. Ancak, günlük ortalama dakikada hedefin biraz altında kalmış. Daha fazla dikkat ve odaklanma ile bu alanı geliştirebilir, daha yüksek bir performans elde edebilir. Alınan ödeme ve puan değerleri ise oldukça başarılı, bu yönüyle takdire değer bir performans sergilemiş.

kardelen:
Kardelen'in toplam arama sayısı ortalamanın biraz altında olsa da ulaşılan sayı ve ulaşma oranı oldukça iyi. Alınan ödeme ve günlük ortalama dakika değerleri de tatmin edici. Puanı ortalamadan yüksek, performansı genel olarak başarılı. Daha fazla arama yaparak toplam arama sayısını artırabilir ve daha iyi bir performans sergileyebilir.

nehir:
Nehir'in bu haftaki performansı genel olarak iyi, ancak ulaşma oranı ortalamanın biraz altında kalmış. Ulaşılan sayı ve alınan ödeme miktarı oldukça başarılı. Günlük ortalama dakika değeri de iyi. Puanı ise ortalamanın altında kalmış, daha fazla çaba gö

In [315]:
file_path_excel = 'C:/Users/DenemeHesap/Desktop/final_df.xlsx' 
final_df.to_excel(file_path_excel, index=False)

In [311]:
final_df

,ekip_no,rehber,toplam_arama,ulasilan,ulasma_orani,ortalama_gunluk_dk,Alınan_Odeme,Puan
0,1,Fadima,1849.000000,612.000000,33.098972,84.906667,1160752.0,8.0
1,1,kardelen,1944.000000,581.000000,29.886831,89.026667,1061950.0,8.0
2,1,nehir,1675.000000,486.000000,29.014925,82.233333,1205643.0,6.0
3,2,nesrin,2169.000000,564.000000,26.002766,98.290000,1092769.0,7.0
4,2,sinem,2080.000000,660.000000,31.730769,98.560000,1073418.0,11.0
5,2,tugce,2048.000000,495.000000,24.169922,58.296667,962424.0,2.0
6,,Ortalama,1960.833333,566.333333,28.984031,85.218889,1092826.0,7.0
